In [1]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input,decode_predictions
from tensorflow.keras.models import Model,model_from_json
import numpy as np
import json

model = ResNet50(weights='imagenet', include_top=True)
model=Model(inputs=model.input,outputs=model.layers[-2].output)
# model.summary()

# img_path = 'elephant.jpg'
# img = image.load_img(img_path, target_size=(224, 224))
# print(type(img))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)

# features = model.predict(x).reshape(-1)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [2]:
model_json=model.to_json()
with open('./models/resnet50.json','w') as f:
    f.write(model_json)

In [8]:
model.save_weights('./models/resnet50.h5')

In [10]:
import time
st=time.time()
with open('./models/resnet50.json','r') as f:
    model_json=f.read()
    
model_new=model_from_json(model_json) 
model_new.load_weights('./models/resnet50.h5')
time.time()-st

52.608970165252686

In [ ]:
with open('fashionmnist_model.json', 'r') as json_file:
    json_savedModel= json_file.read()

In [2]:
import pandas as pd
images=pd.read_csv('./images.csv')[:100]
images=images['link'].to_numpy()
images.shape

(100,)

In [6]:
import multiprocessing
multiprocessing.cpu_count()
for idx,url in enumerate(images):
    print(idx,url)

0 http://assets.myntassets.com/v1/images/style/properties/7a5b82d1372a7a5c6de67ae7a314fd91_images.jpg
1 http://assets.myntassets.com/v1/images/style/properties/4850873d0c417e6480a26059f83aac29_images.jpg
2 http://assets.myntassets.com/v1/images/style/properties/Titan-Women-Silver-Watch_b4ef04538840c0020e4829ecc042ead1_images.jpg
3 http://assets.myntassets.com/v1/images/style/properties/8153dc35d9a5420eeb93922067137db6_images.jpg
4 http://assets.myntassets.com/v1/images/style/properties/Puma-Men-Grey-T-shirt_32668f8a61454d0cc028a808cf21b383_images.jpg
5 http://assets.myntassets.com/v1/images/style/properties/9c1b19682ecf926c296f520d5d6e0972_images.jpg
6 http://assets.myntassets.com/v1/images/style/properties/06e9d4231254fdb2c7fe967ad413ad7b_images.jpg
7 http://assets.myntassets.com/v1/images/style/properties/45ddbc6a15140556214e15923244755b_images.jpg
8 http://assets.myntassets.com/v1/images/style/properties/4ca8848ab441378a392d9d5bf0b0f3c7_images.jpg
9 http://assets.myntassets.com/v1/i

In [10]:
#fitting the nebours

import requests
import pickle
from PIL import Image
features=[]
# list(map(lambda url:requests.get(url, stream=True).raw,images['link'].to_list()))
count=0


for url in images.tolist():
    try:
        im = Image.open(requests.get(url, stream=True).raw).resize((224,224))
        x = image.img_to_array(im)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        count=count+1
        features.append(model.predict(x).reshape(-1))
    except:
        pass
    
pickle_out = open("./models/features.pkl","wb")
pickle.dump(features, pickle_out)
pickle_out.close()
# import json
# with open('./models/features.json','w') as f:
#     json.dump(features,f)

In [ ]:
print(len(features))
from sklearn.neighbors import NearestNeighbors
nei_clf =  NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(np.array(features))

In [11]:
import pickle 


# # Its important to use binary mode 
# knnPickle = open('knn.pkl', 'wb') 

# # source, destination 
# pickle.dump(nei_clf, knnPickle)   
pickle.load(open("./models/features.pkl","rb"))


[]

In [12]:
#predicting

import matplotlib.pyplot as plt
import requests
from PIL import Image

def get_similar(url):

    im = Image.open(requests.get(url, stream=True).raw).resize((224,224))
    x = image.img_to_array(im)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x).reshape(1,-1)
    distances, indices = nei_clf.kneighbors(feature)
    return distances[0],images[indices[0]].tolist()

def show_similar(url):
    
    distances,neighbors = get_similar(url)
    print('executed')
    plt.figure(figsize=[8,7])
    plt.subplot(1,4,1)
    image=Image.open(requests.get(url, stream=True).raw).resize((224,224))
    plt.imshow(image)
    plt.title("Original image")
    
    for i in range(3):
        plt.subplot(1,4,i+2)
        image=Image.open(requests.get(neighbors[i], stream=True).raw).resize((224,224))
        plt.imshow(image)
        plt.title("Dist=%.3f"%distances[i])
    plt.show()

In [ ]:

url=images.tolist()[20]
show_similar(url)

In [16]:
url=images.tolist()[17]
nei_clf=pickle.load(open("./models/knn.pkl","rb"))
im = Image.open(requests.get(url, stream=True).raw).resize((224,224))
x = image.img_to_array(im)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
feature = model.predict(x).reshape(1,-1)
distances, indices = nei_clf.kneighbors(feature)
print(distances)
print(indices[0])

[[ 0.         18.38328336 19.5475347  20.67243055 20.86277566]]
[  17 8060 2168 7038 6579]


In [7]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input,decode_predictions
from tensorflow.keras.models import Model,load_model
import requests
from PIL import Image
import pickle
import pandas as pd

images=pd.read_csv('./images.csv')['link'].to_numpy()

model= load_model('./models/resnet50.h5')
knn_model = pickle.load(open('./models/knn.pkl', 'rb'))

url=images.tolist()[17]
im = Image.open(requests.get(url, stream=True).raw).resize((224,224))
x = image.img_to_array(im)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
feature = model.predict(x).reshape(1,-1)
feature

NameError: name 'np' is not defined